In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.power import FTestAnovaPower

PREPROCESSED_DATA_PATH = 'data\\preprocess'
IPS_PREPROCESS_DATA_PATH = PREPROCESSED_DATA_PATH + '\\IPS'
PSI_PREPROCESS_DATA_PATH = PREPROCESSED_DATA_PATH + '\\PSI'

## Load and process the data

In [ ]:
df_psi = pd.read_csv(PSI_PREPROCESS_DATA_PATH + '\\PSI_.csv')
df_ips = pd.read_csv(IPS_PREPROCESS_DATA_PATH + '\\IPS_.csv')

In [ ]:
df_psi['Group'] = 'PSI'
df_ips['Group'] = 'IPS'

df = pd.concat([df_psi, df_ips], ignore_index=True)
df = df.apply(lambda val: round(100*val/7, 2) if val.name == "PreTest_grade" else val)
df = df.apply(lambda val: round(100*val/8, 2) if val.name == "PostTest_grade" else val)
df.head()

## Basic ploting to understand more our participants

In [ ]:
# Plot age distribution
plt.figure(figsize=(6,4))
sns.countplot(
    data=df,
    x='Age',
    hue='Group',
    palette=['C0','C1']
)
plt.title('Age Distribution by Group')
plt.xlabel('Age (years)')
plt.ylabel('Count')
plt.legend(title='Group')
plt.tight_layout()
plt.show()

In [ ]:
# Plot gender repartition
plt.figure(figsize=(6,4))
sns.countplot(
    data=df,
    x='Gender',
    hue='Group',
    palette=['C0','C1']
)
plt.title('Gender Count by Group')
plt.xlabel('Gender')
plt.ylabel('Count')
plt.legend(title='Group')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
sns.countplot(data=df, x="Section", hue="Group")

plt.title('Repartition of Sections by Group')
plt.xlabel('Section')
plt.ylabel('Number of Participants')
plt.legend(title='Group')
plt.xticks(rotation=90)  # Rotate x labels if they overlap
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
sns.countplot(data=df, x="Lived_region", hue="Group")

plt.title('Repartition of Lived Region by Group')
plt.xlabel('Lived Region')
plt.ylabel('Number of Participants')
plt.legend(title='Group')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
sns.countplot(data=df, x="Program", hue="Group")

plt.title('Repartition of Program by Group')
plt.xlabel('Program')
plt.ylabel('Number of Participants')
plt.legend(title='Group')
plt.xticks(rotation=30)
plt.tight_layout()
plt.show()

## Statistical observations

### Learning gain

In [ ]:
# Compute improvement
df['Learning_Gain'] = df['PostTest_grade'] - df['PreTest_grade']

# Summary stats by condition
summary = ( df
  .groupby('Group')[['PreTest_grade','PostTest_grade','Learning_Gain']]
  .agg(['count','mean','std','median'])
)
summary

In [ ]:
# This was done by Sam using chatGPT

MyTicksNames = ['PreTest', 'PostTest']
MyTicks = np.array([0, 1])
littleSummary = ( df
  .groupby('Group')[['PreTest_grade','PostTest_grade']]
  .agg(['mean'])
)
my_PSI_Results = df.loc[df["Group"] == "PSI", ["PreTest_grade","PostTest_grade"]]
my_IPS_Results = df.loc[df["Group"] == "IPS", ["PreTest_grade","PostTest_grade"]]

# Plotting
fig, ax = plt.subplots(figsize=(6, 4))
myEpsilon = 0.05

# Plotting PSI
ax.plot    (MyTicks - myEpsilon, list(littleSummary.loc["PSI"]), marker='o', label='PSI', color='blue')
ax.boxplot(my_PSI_Results,
           positions = [- myEpsilon, 1 - myEpsilon],
           widths=myEpsilon*2)

# Plotting IPS
ax.plot    (MyTicks + myEpsilon, list(littleSummary.loc["IPS"]), marker='s', label='IPS', color='red')
ax.boxplot(my_IPS_Results,
           positions = [  myEpsilon, 1 + myEpsilon],
           widths=myEpsilon*2)

# Labels and title
ax.set_xticks([0, 1])
ax.set_xticklabels(MyTicksNames)
ax.set_xlabel('Tests')
ax.set_ylabel('Percentages')
ax.set_title('Interaction Plot with mean and boxplots')
ax.legend()

# Display the plot
plt.tight_layout()
plt.show()

In [ ]:
# Box‐plot of improvements
plt.figure(figsize=(6,4))
ax = sns.boxplot(
    data=df,
    x='Group',
    y='Learning_Gain',
    hue='Group',
    palette=['C0','C1']
)
ax.set_title('Score Improvement by Group')
ax.set_ylabel('PostTest – PreTest')
plt.tight_layout()
plt.show()


### Pearson r

In [ ]:
r, p = stats.pearsonr(df_psi['PS time taken (in s)'], df_psi['PostTest_grade'])
print(f"r = {r:.3f}, p = {p:.3f}")
sns.regplot(x='PS time taken (in s)', y='PostTest_grade', data=df_psi)
plt.show()

The p-value is 0.005 < 0.05, thus the effect on the time spend doing the PS part is statisticaly significant on the `PostTest_grade`. We also observe a very strong, positive linear relationship between the two variables(`PostTest_grade` and `PS time taken (in s)`), as showned by the r-value = 0,874. This means that the grade tends to be higher if the participant spended more time in the PS part.

### One-way ANOVA & t-test

In [ ]:
# Fit the model
model = ols('Learning_Gain ~ C(Group)', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

In [ ]:
imp_psi = df.loc[df.Group=='PSI','Learning_Gain']
imp_ips = df.loc[df.Group=='IPS','Learning_Gain']

t_stat, p_val = stats.ttest_ind(imp_psi, imp_ips, equal_var=True)
print(f"t = {t_stat:.3f}, p = {p_val:.3f}")

Since the p-value = 0.744 >> 0.05, we have no evidence of a statistical significance in belonging at one of the two groups, as shown by the one-way ANOVA and the t-test.

### Cohen's d

Cohen’s d is a standardized measure of effect size. It tells you how big the difference between two group means is, in units of their common variability, rather than in the raw score metric (d is unitless).

A rough benchmark originally proposed by Jacob Cohen is:
* d≈0.2 “small” effect
* d≈0.5 “medium” effect
* d≈0.8 “large” effect

In practice, if we got d=0.5 that mean that on average, the mean of Group 1 sits half a standard deviation above the mean of Group 2. If you randomly pick someone from each group, there’s roughly a 64% chance the Group 1 score is higher.

**Why are we using it?**

The p-values tell us whether an effect is likely “real” (statistically significant), but it depend heavily on the sample size. Cohen’s d tells you how large that effect is, independently of sample size.
Nevertheless, for small samples like our one, Cohen’s d can overestimate the “true” population effect size.

Futhermore, this value will be useful when conducting a Power Analysis.

In [ ]:
def cohens_d(x, y):
    nx, ny = len(x), len(y)
    sx = x.std(ddof=1); sy = y.std(ddof=1)
    pooled_sd = np.sqrt(((nx-1)*sx**2 + (ny-1)*sy**2) / (nx+ny-2))
    return (x.mean() - y.mean()) / pooled_sd
    
d = cohens_d(imp_psi, imp_ips)
print(f"Cohen’s d (PSI vs IPS learning gain): {d:.3f}")

On average, the PSI group improved slightly less than the IPS group. But the effect seems negligible according to Cohen benchmark. The two methods produced almost identical mean gains in our sample. This also support our previous observation made with the one-way ANOVA.

### Power Analysis

In [ ]:
effect_size = abs(d) / 2 # Convert Cohen's d to effect size for ANOVA
nobs = 16
alpha = 0.05

In [ ]:
analysis = FTestAnovaPower()
power = analysis.power(effect_size, nobs, alpha)
print(f"Achieved power: {power:.3f}")

In [ ]:
required_n = analysis.solve_power(effect_size, None, alpha, 0.8)
print(f"Required total sample size for 80% power: {required_n:.0f}")

The power analysis showes an extremely low achived power (0.058). This indicate a highly probable Type II error (failure to detect a true effect if one exists). To achieve 80% power for an effect of this size, an estimated 1572 participants would be required. Therefore, the current study is underpowered.